# Installation

The following instructions were prepared using

In [1]:
versioninfo()

Julia Version 1.10.4
Commit 48d4fd48430 (2024-06-04 10:41 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 16 × AMD Ryzen 7 7840U w/ Radeon  780M Graphics
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, znver3)
Threads: 8 default, 0 interactive, 4 GC (on 16 virtual cores)
Environment:
  JULIA_NUM_THREADS = 8


Before exploring the notebook you need to clone the main repository:

```
git clone https://github.com/kalmarek/2306.12358.git
```

This notebook should be located in `2306.12358/notebooks` directory.

In the main directory (`2306.12358`) you should run the following code in julias REPL console to instantiate the environment for computations:

```julia
using Pkg
Pkg.activate(".")
Pkg.instantiate()
```

(this needs to be done once per installation).

Instantiation should install (among others):
* JuMP package for mathematical programming (https://jump.dev/),
* SCS solver (https://github.com/cvxgrp/scs),
* COSMO solver (https://github.com/oxfordcontrol/COSMO.jl),
* and `IntervalArithmetic.jl` package from ValidatedNumerics (https://juliaintervals.github.io/).

The environment uses [`Groups.jl`](https://github.com/kalmarek/Groups.jl), [`StarAlgebras.jl`](https://github.com/kalmarek/StarAlgebras.jl/), [`SymbolicWedderburn.jl`](https://github.com/kalmarek/SymbolicWedderburn.jl/) and [`PropertyT.jl`](https://github.com/kalmarek/PropertyT.jl/) packages.

A jupyter server may then be launched within the directory `2306.12358` by issuing from julia command-line (REPL) the following commands.

```julia
using Pkg
Pkg.activate(".")
using IJulia
notebook(dir=pwd())
```

During the first run, the user may be asked for installation of Jupyter program (a server for running this notebook) within miniconda environment, which will happen automatically after confirmation. To execute the commands in the notebook, one needs to navigate to notebooks subdirectory of `2306.12358` and click either of the notebooks.

> Code "cells" have a label, e.g. In [1]: on their left margin. Click somewhere in the proximity of the label to select a cell. The selected cell is run by either clicking the `Run` button in the top toolbar, or by pressing `Shift+Enter`.



In [2]:
using Pkg

In [3]:
Pkg.activate(joinpath(@__DIR__, ".."))

  Activating project at `~/Mathematics/Research/Property (T)/Chevalley/2306.12358`


In [4]:
Pkg.status()

Status `~/Mathematics/Research/Property (T)/Chevalley/2306.12358/Project.toml`
  [c7e460c6] ArgParse v1.2.0
  [1e616198] COSMO v0.8.9
  [5d8bd718] Groups v0.8.0
  [7073ff75] IJulia v1.25.0
  [4076af6c] JuMP v1.22.2
  [03b72c93] PropertyT v0.6.0 `https://github.com/kalmarek/PropertyT.jl#master`
  [c946c3f1] SCS v2.0.0
⌃ [3f2553a9] SCS_MKL_jll v3.2.4+1 ⚲
  [ade2ca70] Dates
  [37e2e46d] LinearAlgebra
Info Packages marked with ⌃ have new versions available and may be upgradable.


In [5]:
using Groups
import Groups.MatrixGroups

In [6]:
using PropertyT

# $\texttt{A}_\texttt{2}$ graded $\operatorname{Adj}$

We wish to prove
> **Theorem 3.6** Let $G = \operatorname{SL}_3(\mathbb{Z})$ be the universal Chevalley group over $\mathbb{Z}$ of type $\texttt{A}_\texttt{2}$ endowed with the set of Steinberg generators $S$. Let $V$ denote the ambient vector space of the root system. Then $$\operatorname{Adj}_V −\lambda \Delta_V ⩾_R 0$$
whenever $(\lambda, R) \in (0.158606, 2), (0.273954, 3)$.

Below we will prove the theorem for $R = 2$. Proving it for $R = 3$ requires changing `HALFRADIUS` to $3$ below and a bit of patience.

In [7]:
N = 3
G = MatrixGroups.SpecialLinearGroup{N}(Int8)

special linear group of 3×3 matrices over Int8

In [8]:
g = gens(G, 1)

E₁₂ ∈ SL{3,Int8}
 1  1  0
 0  1  0
 0  0  1

In [9]:
HALFRADIUS = 2
RG, S, sizes = @time PropertyT.group_algebra(G, halfradius = HALFRADIUS);

  0.004542 seconds (135.29 k allocations: 5.381 MiB)
  0.000827 seconds (21.64 k allocations: 846.062 KiB)
  0.667067 seconds (1.62 M allocations: 106.888 MiB, 2.90% gc time, 99.09% compilation time)


[ Info: generating wl-metric ball of radius 4
[ Info: sizes = [13, 121, 883, 5455]
[ Info: computing the *-algebra structure for G


In [10]:
S

12-element Vector{FPGroupElement{Groups.MatrixGroups.SpecialLinearGroup{3, Int8, DataType, Groups.MatrixGroups.ElementaryMatrix{3, Int8}}, KnuthBendix.Words.Word{UInt8}}}:
 E₁₂
 E₁₃
 E₂₁
 E₂₃
 E₃₁
 E₃₂
 E₁₂^-1
 E₁₃^-1
 E₂₁^-1
 E₂₃^-1
 E₃₁^-1
 E₃₂^-1

In [11]:
Δ = RG(length(S)) - sum(RG(s) for s in S)

12·(id) -1·E₁₂ -1·E₁₃ -1·E₂₁ -1·E₂₃ -1·E₃₁ -1·E₃₂ -1·E₁₂^-1 -1·E₁₃^-1 -1·E₂₁^-1 -1·E₂₃^-1 -1·E₃₁^-1 -1·E₃₂^-1

## Grading

Let us define $\texttt{A}_\texttt{2}$ grading for $(G, S)$. That is, to every generator $s \in S$ we need to assign a root in $\texttt{A}_\texttt{2}$ in a compatible fashion. For the precise description of this grading see **Example 3.2 (1)** of the accompanying paper.  

In [12]:
g

E₁₂ ∈ SL{3,Int8}
 1  1  0
 0  1  0
 0  0  1

In [13]:
PropertyT.grading(g)

Root in ℝ^3 of length √2
[1, -1, 0]

In [14]:
Δs = PropertyT.laplacians(
    RG,
    S,
    x -> (gx = PropertyT.grading(x); Set([gx, -gx])),
);

Here `Δs` is just a map from lines in the root system $\Omega = \texttt{A}_{\texttt{2}}$ to the corresponding Laplacians; e.g. below we can see that to the line through `α = [1, -1, 0]` and `-α` (and the origin) we assign
$$ \Delta_{Lα} = 4 - E_{1,2} - E_{2,1} - E_{1,2}^{-1} - E_{2,1}^{-1}.$$ 

In [15]:
using PropertyT.Roots # for nicer printing
α = Root([1,-1,0])
Lα = Set([α, -α])
Δs[Lα]

4·(id) -1·E₁₂ -1·E₂₁ -1·E₁₂^-1 -1·E₂₁^-1

Following the definition of $\operatorname{Adj}$ we define
$$ \operatorname{Adj}_{\texttt{A}_\texttt{2}} = 
\prod_{
    \langle L\alpha, L\beta \rangle \cap \Omega \cong \texttt{A}_{\texttt{2}}
} \Delta_{L\alpha} \Delta_{L\beta} $$

In [16]:
AdjA₂ = PropertyT.Adj(Δs, :A₂)

96·(id) -16·E₁₂ -16·E₁₃ -16·E₂₁ -16·E₂₃ -16·E₃₁ -16·E₃₂ -16·E₁₂^-1 -16·E₁₃^-1 -16·E₂₁^-1 -16·E₂₃^-1 -16·E₃₁^-1 -16·E₃₂^-1 +2·E₁₂*E₁₃ +1·E₁₂*E₂₃ +1·E₁₂*E₃₁ +2·E₁₂*E₃₂ +2·E₁₂*E₁₃^-1 +1·E₁₂*E₂₃^-1 +1·E₁₂*E₃₁^-1 +2·E₁₂*E₃₂^-1 +1·E₁₃*E₂₁ +2·E₁₃*E₂₃ +1·E₁₃*E₃₂ +2·E₁₃*E₁₂^-1 +1·E₁₃*E₂₁^-1 +2·E₁₃*E₂₃^-1 +1·E₁₃*E₃₂^-1 +1·E₂₁*E₁₃ +2·E₂₁*E₂₃ +2·E₂₁*E₃₁ +1·E₂₁*E₃₂ +1·E₂₁*E₁₃^-1 +2·E₂₁*E₂₃^-1 +2·E₂₁*E₃₁^-1 +1·E₂₁*E₃₂^-1 +1·E₂₃*E₁₂ +1·E₂₃*E₃₁ +1·E₂₃*E₁₂^-1 +2·E₂₃*E₁₃^-1 +2·E₂₃*E₂₁^-1 +1·E₂₃*E₃₁^-1 +1·E₃₁*E₁₂ +1·E₃₁*E₂₃ +2·E₃₁*E₃₂ +1·E₃₁*E₁₂^-1 +2·E₃₁*E₂₁^-1 +1·E₃₁*E₂₃^-1 +2·E₃₁*E₃₂^-1 +1·E₃₂*E₁₃ +1·E₃₂*E₂₁ +2·E₃₂*E₁₂^-1 +1·E₃₂*E₁₃^-1 +1·E₃₂*E₂₁^-1 +2·E₃₂*E₃₁^-1 +1·E₁₂^-1*E₂₃ +1·E₁₂^-1*E₃₁ +2·E₁₂^-1*E₁₃^-1 +1·E₁₂^-1*E₂₃^-1 +1·E₁₂^-1*E₃₁^-1 +2·E₁₂^-1*E₃₂^-1 +1·E₁₃^-1*E₂₁ +1·E₁₃^-1*E₃₂ +1·E₁₃^-1*E₂₁^-1 +2·E₁₃^-1*E₂₃^-1 +1·E₁₃^-1*E₃₂^-1 +1·E₂₁^-1*E₁₃ +1·E₂₁^-1*E₃₂ +1·E₂₁^-1*E₁₃^-1 +2·E₂₁^-1*E₂₃^-1 +2·E₂₁^-1*E₃₁^-1 +1·E₂₁^-1*E₃₂^-1 +1·E₂₃^-1*E₁₂ +1·E₂₃^-1*E₃₁ +1·E₂₃^-1*E₁₂^-1 +1·E₂₃^-1*E₃₁^-1 +1·E₃₁^-1*E

It is not hard to see that for $\Omega = \texttt{A}_{\texttt{2}}$ 
 * we are simply looking at products of all $\Delta_{L\alpha}$ and $\Delta_{L\beta}$ where $L\alpha \neq L\beta$, and
 * that the new definition agrees with the definition of $\operatorname{Adj_3}$ from [On property (T) for $\operatorname{Aut}(F_n)$ and $\operatorname{SL}_n(\mathbb{Z})$](https://arxiv.org/abs/1812.03456):

In [17]:
AdjA₂ == Δ^2 - sum(Δs[Lα]^2 for Lα in keys(Δs))

true

In [18]:
let (sq,adj,op) = PropertyT.SqAdjOp(RG, N)
    AdjA₂ == adj && AdjA₂ == Δ^2 - sq
end

true

## Optimization problem
To prove **Theorem 3.6** we need to rewrite $\operatorname{Adj}_{\texttt{A}_\texttt{2}} - \lambda \Delta$ as a sum of squares in the group algebra $\mathbb{R}G$.

Below `PropertyT.sos_problem_primal` takes $x$ and $u$, two elements of a *-algebra $\mathcal{A}$ (e.g. a group ring) and returns an optimization problem where we try to 

$$ \begin{aligned}
\max\quad &\lambda \\
\text{subject to:}\quad & \operatorname{x} - \lambda\cdot\operatorname{u} = \sum_i^n \xi_i^* \xi_i\\
\quad & \xi_i \in \mathcal{A}.
\end{aligned}
$$

This is internally obtained by translating this problem into a problem of semi-definite optimization as follows:

$$ \begin{aligned}
\max\quad &\lambda \\
\text{subject to:}\quad & (\operatorname{x} - \lambda\cdot\operatorname{u})(t) = \langle A_t, P\rangle, \quad t \in \operatorname{Ball}(S, 2R)\\
& P \succeq 0,\\
\end{aligned}
$$

for some Boolean matrices $A_t \in \operatorname{Mat}_{n,n}$ where $n = \left|\operatorname{Ball}(S, R)\right|$. These matrices are defined by the multiplicative structure on $\mathcal{A}$, i.e.

$$ \left(A_t\right)_{a,b} = \left\{\begin{align}1 \quad & \text{when }a^* \cdot b == t\\ 0 \quad & \text{otherwise}\end{align}\right.$$

The translation is done simply by realizing that since $P$ is positive semi-definite, there exists a real root $Q$ of $P$, i.e. $Q^T\cdot Q = P$. Then $\xi_i$ is obtained by treating the $i$-th column of $Q$ as a vector of coefficients w.r.t. basis $\operatorname{Ball}(S, R)$ and extending $\xi_i$ to $0$ elsewhere.

The problem with this formulation is that the constraint $P \succeq 0$ quickly becomes numerically untractable.

In [19]:
@time model = PropertyT.sos_problem_primal(AdjA₂, Δ, augmented = true)

  1.384055 seconds (3.69 M allocations: 257.542 MiB, 6.65% gc time, 95.48% compilation time: <1% of which was recompilation)


A JuMP Model
Maximization problem with:
Variables: 7382
Objective function type: JuMP.VariableRef
`JuMP.AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 5455 constraints
`Vector{JuMP.VariableRef}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: P, psd, λ

### Symmetry reduction

To simplify the optimization problem we will use Wedderburn decomposition of the (finite dimensional) subspace $E_R < \mathbb{R}G$ spanned by all elements in $\operatorname{Ball}(S, R)$, the ball of radius $R$:

In [20]:
import PropertyT.SA as StarAlgebras
import PropertyT.SW as SymbolicWedderburn
using PropertyT.PG # PermutationGroups

In [21]:
wd = let RG = RG, N = N
    G = StarAlgebras.object(RG)
    P = PermGroup(perm"(1,2)", Perm(circshift(1:N, -1)))
    Weyl = Groups.Constructions.WreathProduct(PermGroup(perm"(1,2)"), P)
    act = PropertyT.action_by_conjugation(G, Weyl)

    @time SymbolicWedderburn.WedderburnDecomposition(
        Float64,
        Weyl,
        act,
        StarAlgebras.basis(RG),
        StarAlgebras.Basis{UInt16}(@view StarAlgebras.basis(RG)[1:sizes[HALFRADIUS]]),
    )
end
@info wd

  8.269365 seconds (14.91 M allocations: 1007.374 MiB, 2.61% gc time, 230.44% compilation time: <1% of which was recompilation)


┌ Info: Wedderburn Decomposition into 247 orbits and 5 simple summands of sizes
└ [9, 6, 14, 14, 12]


Now instead of

 * $121 \times 121$ positive-semidefinite (PSD) constraint $P \succeq 0$ and
 * $5455$ linear constraints $(x-\lambda\cdot u)(t) = \langle A_t, P\rangle$ (i.e. one for every element $t\in \operatorname{Ball}(S, 2R)$

our symmetrized problem will have

* $5$ PSD constraints (of sizes $14 \times 14$, $14 \times 14$, $12\times 12$, $9\times 9$ and $6 \times 6$),
* $247$ linear constraints i.e. one for every orbit of `Weyl` action on $\operatorname{Ball}(S, 2R)$.

In [22]:
@time model, varP = PropertyT.sos_problem_primal(AdjA₂, Δ, wd; augmented = true);
model

  1.969343 seconds (4.05 M allocations: 290.117 MiB, 3.19% gc time, 288.86% compilation time)


A JuMP Model
Maximization problem with:
Variables: 355
Objective function type: JuMP.VariableRef
`JuMP.AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 246 constraints
`Vector{JuMP.VariableRef}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 5 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: λ

## Solving the problem numerically

Let us bring a __solver__ i.e. a piece of software specialized to solve conic (in this case) optimization problems, such as the one defined by our sum of squares decomposition. We will use `scs` [Splitting Conic Solver](https://github.com/cvxgrp/scs).

In [23]:
warm = nothing

In [24]:
using JuMP
include(joinpath(@__DIR__, "..", "src", "optimizers.jl"));
with_optimizer = scs_optimizer(;
    eps = 3e-9,
    max_iters = 20_000,
    accel = 50,
    alpha = 1.95,
);

In [25]:
status, warm = PropertyT.solve(
        model,
        with_optimizer,
        warm,
    );
@info "Optimization has finished with" status

------------------------------------------------------------------
	       SCS v3.2.4 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 355, constraints m: 600
cones: 	  z: primal zero / dual free vars: 246
	  s: psd vars: 354, ssize: 5
settings: eps_abs: 3.0e-09, eps_rel: 3.0e-09, eps_infeas: 1.0e-07
	  alpha: 1.95, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 20000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 50, acceleration_interval: 10
	  compiled with openmp parallelization enabled
lin-sys:  sparse-direct-mkl-pardiso
	  nnz(A): 4119, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 4.86e+01  1.00e+00  3.52e+02 -2.03e+02  1.00e-01  1.06e-02 
   250| 1.41e-02  4.06e-04  3.71e-02 -4.02e

┌ Info: Optimization has finished with
└   status = OPTIMAL::TerminationStatusCode = 1


> **Note**: Solving this problem for `HALFRADIUS=3` will require
> * setting `max_iters` to `1_000_000` in `scs_optimizer` above,
> * running the optimization for about `6` hours.

### Reconstructing and certifying the solution
Since we symmetrized our problem, we don't have direct access to `P` or `Q` as defined above. So first we need to reconstruct `Q` from the action of `Weyl` on $\operatorname{Ball}(S, R)$:

In [26]:
@info "reconstructing the solution"
Q = @time let wd = wd, Ps = [JuMP.value.(P) for P in varP]
    Qs = real.(sqrt.(Ps))
    PropertyT.reconstruct(Qs, wd)
end

[ Info: reconstructing the solution


 14.594043 seconds (23.98 M allocations: 1.442 GiB, 2.84% gc time, 101.07% compilation time)


121×121 Matrix{Float64}:
 0.0   0.0          0.0          0.0         …   0.0           0.0
 0.0   1.98068      0.117884    -0.172455        0.0154775    -0.0299589
 0.0   0.117884     1.98068      0.0323778       0.00329992    0.0157457
 0.0  -0.172455     0.0323778    1.98068         0.00329992    0.0157457
 0.0   0.0323778    0.117884     0.117884        0.00339668   -0.0112047
 0.0   0.0323778   -0.172455     0.117884    …   0.0154775    -0.0299589
 0.0   0.117884     0.0323778    0.0323778       0.00354317    0.00694926
 0.0  -0.233031     0.117884    -0.170735        0.0154775    -0.0299589
 0.0   0.117884    -0.233031     0.0323778       0.00329992    0.0157457
 0.0  -0.170735     0.0323778   -0.233031        0.00329992    0.0157457
 0.0   0.0323778    0.117884     0.117884    …   0.0168681     0.00878248
 0.0   0.0323778   -0.170735     0.117884        0.0154775    -0.0299589
 0.0   0.117884     0.0323778    0.0323778       0.0121586     0.0330342
 ⋮                            

> Note: since we _augmented_ our problem, the value at identity of each $\xi_i \in IG$ (the augmentation ideal) is determined by the others and hence we dropped those from the actual optimization.

As can be seen, $\xi_i$ is supported on the first 121 elements (which is the size of $\operatorname{Ball}(S, 2)$) and extended to zeros elsewhere. Let us compute the corresponding sum of squares decomposition and compare it with $\operatorname{Adj}_{\texttt{A}_{\texttt{2}}} - \lambda \Delta$.

In [27]:
import LinearAlgebra

In [28]:
sos_decomposition = PropertyT.compute_sos(RG, Q, augmented=true)
λ = JuMP.objective_value(model)
residual = (AdjA₂ - λ*Δ) - sos_decomposition
LinearAlgebra.norm(residual, 1)

4.210657152326916e-7

The residual (although small) is not exactly $0$. This is due to finite precision of numerical computations as well as the iterative nature of the solver. To amend the situation we need a certified upper bound on `norm(residual, 1)`, the $\ell_1$-norm of the residual, and a lemma by T.Netzer and A.Thom:

> **Lemma** (Lemma 2.1 of [Netzer-Thom](https://arxiv.org/abs/1411.2488), Lemma 4.10 of [Kaluba-Kielak-Nowak](https://arxiv.org/abs/1812.03456)):
>
> Suppose that $r \in IG$ is hermitian (i.e. $r^* = r$) and is supported on $\operatorname{Ball}(S, 2R)$. Then
>
> $$r + \varepsilon \Delta = \sum_{i}\eta_i^* \eta_i$$
> 
> for every $\varepsilon \geqslant 2^{2\lceil\log_2 R\rceil}\|r\|_1$. If $S$ contains no involutions then it suffices to have $\varepsilon \geqslant 2^{2\lceil\log_2 R\rceil-1}\|r\|_1$.



To achieve certified results in finite precision computations we use interval arithmetic:

In [29]:
using PropertyT.IntervalArithmetic

In [30]:
ℓ₁norm_residual = let Q = Interval.(Q), λ = Interval(λ)
    sos_decomposition = PropertyT.compute_sos(RG, Q, augmented=true)
    residual = (AdjA₂ - λ*Δ) - sos_decomposition
    LinearAlgebra.norm(residual, 1)
end

[4.20946e-07, 4.21191e-07]

In [31]:
certified_λ = Interval(λ) - 2^(2*ceil(log2(Interval(HALFRADIUS)))-1)*ℓ₁norm_residual

[0.158605, 0.158606]

and therefore we proved that $\operatorname{Adj}_{\texttt{A}_\texttt{2}} - \lambda \Delta \geqslant 0$ for every $\lambda$ smaller than

In [32]:
inf(certified_λ)

0.15860560506669596

> **Note**: All of this could have been achieved with a single call to the following convenience function:

In [33]:
@info "certifying the solution"
certified, λ = PropertyT.certify_solution(
    AdjA₂,
    Δ,
    JuMP.objective_value(model),
    Q;
    halfradius = HALFRADIUS,
    augmented = true,
)

if certified && λ > 0
    @info "Certified result: Adj_A₂ is positive" inf(λ)
else
    @info "Could NOT certify the positivity of Adj_A₂" certified λ
end

[ Info: certifying the solution
┌ Info: Checking in Float64 arithmetic with
└   λ = 0.1586064474476758


  0.000266 seconds (6 allocations: 157.281 KiB)


┌ Info: Numerical metrics of the obtained SOS:
│ ɛ(elt - λu - ∑ξᵢ*ξᵢ) ≈ 1.21408e-12
│ ‖elt - λu - ∑ξᵢ*ξᵢ‖₁ ≈ 4.21066e-7
└  λ ≈ 0.15860476318481487
┌ Info: Checking in Interval{Float64} arithmetic with
└   λ_int = [0.158606, 0.158607]


  0.214073 seconds (133.42 k allocations: 11.150 MiB, 99.17% compilation time)


┌ Info: Numerical metrics of the obtained SOS:
│ ɛ(elt - λu - ∑ξᵢ*ξᵢ) ∈ [-1.15174e-10, 1.11503e-10]
│ ‖elt - λu - ∑ξᵢ*ξᵢ‖₁ ∈ [4.20955e-07, 4.21183e-07]
└  λ ∈ [0.158604, 0.158605]
┌ Info: Certified result: Adj_A₂ is positive
└   inf(λ) = 0.1586047627191617


# Computation of $\gamma(\Omega)$

Let $\Omega$ be one of the irreducible root systems $E_6$, $E_7$, $E_8$. We wish to confirm here the computations of **Table 3.9** for the generating sets and $\gamma(\Omega)$.

In [34]:
countmap(v) = countmap(identity, v)
function countmap(f, v)
    counts = Dict{eltype(f(first(v))),Int}()
    for x in v
        fx = f(x)
        counts[fx] = get!(counts, fx, 0) + 1
    end
    return counts
end

countmap (generic function with 2 methods)

In [35]:
# define the natural permutation action on tuples by permuting their entries:
Base.:^(t::NTuple{N}, p::PermutationGroups.AbstractPermutation) where N = ntuple(i->t[i^p], N)

## $E_8$
We start with the definition of $E_8$ as a specific set of vectors in $\mathbb{R}^8$. Namely
1. as long roots we take take all possible permutations of
    * $( 1, 1, 0,0,0,0,0,0)$,
    * $(-1, 1, 0,0,0,0,0,0)$ and
    * $(-1, -1,0,0,0,0,0,0)$.
2. as short roots we take all possible combinations of **even sign changes** applied to the vector
   $\left(\frac{1}{2},\frac{1}{2},\frac{1}{2},\frac{1}{2},\frac{1}{2},\frac{1}{2},\frac{1}{2},\frac{1}{2}\right)$.
   
Alltogether, there are $240$ such vectors.

In [36]:
E₈ = let Σ = PermutationGroups.PermGroup(perm"(1,2,3,4,5,6,7,8)", perm"(1,2)")
    
    long = let x = (1, 1, 0, 0, 0, 0, 0, 0) .// 1
        PropertyT.Roots.Root.(
            union(
                (x^g for g in Σ),
                ((x .* (-1, 1, 1, 1, 1, 1, 1, 1))^g for g in Σ),
                ((-1 .* x)^g for g in Σ),
            ),
        )
    end

    signs = collect(
        p for p in Iterators.product(fill([-1, +1], 8)...) if
        iseven(count(==(-1), p))
    )
    halfs = let x = (1, 1, 1, 1, 1, 1, 1, 1) .// 2
        rts = unique(PropertyT.Roots.Root(x .* sgn) for sgn in signs)
    end

    union(long, halfs)
end

240-element Vector{Root{8, Rational{Int64}}}:
 Root Rational{Int64}[1, 1, 0, 0, 0, 0, 0, 0]
 Root Rational{Int64}[1, 0, 1, 0, 0, 0, 0, 0]
 Root Rational{Int64}[1, 0, 0, 0, 0, 0, 0, 1]
 Root Rational{Int64}[1, 0, 0, 1, 0, 0, 0, 0]
 Root Rational{Int64}[1, 0, 0, 0, 1, 0, 0, 0]
 Root Rational{Int64}[1, 0, 0, 0, 0, 1, 0, 0]
 Root Rational{Int64}[1, 0, 0, 0, 0, 0, 1, 0]
 Root Rational{Int64}[0, 0, 0, 0, 0, 0, 1, 1]
 Root Rational{Int64}[0, 0, 0, 1, 0, 0, 1, 0]
 Root Rational{Int64}[0, 0, 0, 0, 1, 0, 1, 0]
 Root Rational{Int64}[0, 0, 0, 0, 0, 1, 1, 0]
 Root Rational{Int64}[0, 0, 1, 0, 0, 0, 1, 0]
 Root Rational{Int64}[0, 0, 0, 0, 0, 1, 0, 1]
 ⋮
 Root Rational{Int64}[-1//2, -1//2, -1//2, 1//2, -1//2, 1//2, 1//2, 1//2]
 Root Rational{Int64}[1//2, 1//2, -1//2, 1//2, -1//2, 1//2, 1//2, 1//2]
 Root Rational{Int64}[1//2, -1//2, 1//2, 1//2, -1//2, 1//2, 1//2, 1//2]
 Root Rational{Int64}[-1//2, 1//2, 1//2, 1//2, -1//2, 1//2, 1//2, 1//2]
 Root Rational{Int64}[-1//2, -1//2, -1//2, -1//2, 1//2, 1//2, 1

Let us classify all irreducible and admissible (see **Definition 2.5**) subspaces $W<\mathbb{R}^8$. By symmetry, we may assume that all of these contain a single (here: the first) root $\alpha \in \Omega$:

In [37]:
admissible_types_E₈ = let Ω = E₈, α = first(Ω)
    countmap([
        Roots.classify_sub_root_system(Ω, α, γ) for
        γ in Ω if !Roots.isproportional(α, γ)
    ])
end

Dict{Any, Int64} with 2 entries:
  :A₂             => 112
  Symbol("A₁×A₁") => 126

Since every admissible subspace containing $\alpha$ has been counted $4$ times we obtain

In [38]:
@assert rem(admissible_types_E₈[:A₂], 4) == 0
γE₈ = admissible_types_E₈[:A₂] ÷ 4

28

## $E_7$
Let $V_0 = \left\{x \in \mathbb{R}^8 \colon \sum_i x_i = 0 \right\}$.
To reuse the data we have already computed we define $E_7 = E_8 \cap V_0$ and perform analogous computations for types of irreducible and admissible subspaces. There are $126$ roots in $E_7$:

In [39]:
E₇ = filter(r -> iszero(sum(r.coord)), E₈)

126-element Vector{Root{8, Rational{Int64}}}:
 Root Rational{Int64}[-1, 1, 0, 0, 0, 0, 0, 0]
 Root Rational{Int64}[-1, 0, 1, 0, 0, 0, 0, 0]
 Root Rational{Int64}[-1, 0, 0, 0, 0, 0, 0, 1]
 Root Rational{Int64}[-1, 0, 0, 1, 0, 0, 0, 0]
 Root Rational{Int64}[-1, 0, 0, 0, 1, 0, 0, 0]
 Root Rational{Int64}[-1, 0, 0, 0, 0, 1, 0, 0]
 Root Rational{Int64}[-1, 0, 0, 0, 0, 0, 1, 0]
 Root Rational{Int64}[1, 0, 0, 0, 0, 0, 0, -1]
 Root Rational{Int64}[1, 0, 0, -1, 0, 0, 0, 0]
 Root Rational{Int64}[1, 0, 0, 0, -1, 0, 0, 0]
 Root Rational{Int64}[1, 0, 0, 0, 0, -1, 0, 0]
 Root Rational{Int64}[1, 0, -1, 0, 0, 0, 0, 0]
 Root Rational{Int64}[1, 0, 0, 0, 0, 0, -1, 0]
 ⋮
 Root Rational{Int64}[1//2, -1//2, -1//2, 1//2, -1//2, -1//2, 1//2, 1//2]
 Root Rational{Int64}[-1//2, 1//2, -1//2, 1//2, -1//2, -1//2, 1//2, 1//2]
 Root Rational{Int64}[-1//2, -1//2, 1//2, 1//2, -1//2, -1//2, 1//2, 1//2]
 Root Rational{Int64}[1//2, -1//2, -1//2, -1//2, 1//2, -1//2, 1//2, 1//2]
 Root Rational{Int64}[-1//2, 1//2, -1//2, -1

In [40]:
admissible_types_E₇ = let Ω = E₇, α = first(Ω)
    countmap([
        Roots.classify_sub_root_system(Ω, α, γ) for
        γ in Ω if !Roots.isproportional(α, γ)
    ])
end

Dict{Any, Int64} with 2 entries:
  :A₂             => 64
  Symbol("A₁×A₁") => 60

In [41]:
@assert rem(admissible_types_E₇[:A₂], 4) == 0
γE₇ = admissible_types_E₇[:A₂] ÷ 4

16

## $E_6$
Finally we define $E_6 = E_8 \cap V_1$ where $V_1 = \left\{x \in \mathbb{R}^8 \colon x_8 = x_7 = x_6 \right\}$.
There are $72$ vectors in $E_6$:

In [42]:
E₆ = filter(r -> r.coord[end] == r.coord[end-1] == r.coord[end-2], E₈)

72-element Vector{Root{8, Rational{Int64}}}:
 Root Rational{Int64}[1, 1, 0, 0, 0, 0, 0, 0]
 Root Rational{Int64}[1, 0, 1, 0, 0, 0, 0, 0]
 Root Rational{Int64}[1, 0, 0, 1, 0, 0, 0, 0]
 Root Rational{Int64}[1, 0, 0, 0, 1, 0, 0, 0]
 Root Rational{Int64}[0, 0, 0, 1, 1, 0, 0, 0]
 Root Rational{Int64}[0, 0, 1, 1, 0, 0, 0, 0]
 Root Rational{Int64}[0, 0, 1, 0, 1, 0, 0, 0]
 Root Rational{Int64}[0, 1, 0, 1, 0, 0, 0, 0]
 Root Rational{Int64}[0, 1, 0, 0, 1, 0, 0, 0]
 Root Rational{Int64}[0, 1, 1, 0, 0, 0, 0, 0]
 Root Rational{Int64}[-1, 1, 0, 0, 0, 0, 0, 0]
 Root Rational{Int64}[-1, 0, 1, 0, 0, 0, 0, 0]
 Root Rational{Int64}[-1, 0, 0, 1, 0, 0, 0, 0]
 ⋮
 Root Rational{Int64}[-1//2, -1//2, -1//2, 1//2, -1//2, 1//2, 1//2, 1//2]
 Root Rational{Int64}[1//2, 1//2, -1//2, 1//2, -1//2, 1//2, 1//2, 1//2]
 Root Rational{Int64}[1//2, -1//2, 1//2, 1//2, -1//2, 1//2, 1//2, 1//2]
 Root Rational{Int64}[-1//2, 1//2, 1//2, 1//2, -1//2, 1//2, 1//2, 1//2]
 Root Rational{Int64}[-1//2, -1//2, -1//2, -1//2, 1//2, 1//2,

In [43]:
admissible_types_E₆ = let Ω = E₆, α = first(Ω)
    countmap([
        Roots.classify_sub_root_system(Ω, α, γ) for
        γ in Ω if !Roots.isproportional(α, γ)
    ])
end

Dict{Any, Int64} with 2 entries:
  :A₂             => 40
  Symbol("A₁×A₁") => 30

In [44]:
@assert rem(admissible_types_E₆[:A₂], 4) == 0
γE₆ = admissible_types_E₆[:A₂] ÷ 4

10

Thus the **Table 3.9** has been computed as follows 

In [45]:
κ(λ, S) = sqrt(2λ/S)
λA₂ = @interval(0.273954)

[0.273953, 0.273955]

In [46]:
for (i, (E, γ)) in enumerate([(E₆, γE₆), (E₇, γE₇), (E₈, γE₈)])
    n = i+5
    λ = γ*λA₂
    lS = 2*length(E)
    λ_inf = round(inf(λ), RoundDown, digits=5)
    κ_inf = round(inf(κ(λ, lS)), RoundDown, digits=5)
    @info "$(Symbol(:E, n))" (lS, γ, λ_inf, κ_inf)
end
    

┌ Info: E6
└   (lS, γ, λ_inf, κ_inf) = (144, 10, 2.73953, 0.19506)
┌ Info: E7
└   (lS, γ, λ_inf, κ_inf) = (252, 16, 4.38326, 0.18651)
┌ Info: E8
└   (lS, γ, λ_inf, κ_inf) = (480, 28, 7.67071, 0.17877)


In [47]:
using Dates
Dates.now()

2024-07-08T14:46:41.131

In [48]:
versioninfo()

Julia Version 1.10.4
Commit 48d4fd48430 (2024-06-04 10:41 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 16 × AMD Ryzen 7 7840U w/ Radeon  780M Graphics
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, znver3)
Threads: 8 default, 0 interactive, 4 GC (on 16 virtual cores)
Environment:
  JULIA_NUM_THREADS = 8
